In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Data Loading
df = pd.read_csv(r'crop_yield.csv')
print("Shape of the DataFrame:", df.shape)
print("\nData Types:\n", df.info())
print("\nMissing Values:\n", df.isnull().sum())

# Data Preprocessing
df['Yield'] = winsorize(df['Yield'], limits=[0, 0.05])
categorical_cols = ['Crop', 'Season', 'State']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
numerical_cols = ['Area', 'Production', 'Annual_Rainfall', 'Fertilizer', 'Pesticide', 'Yield']
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.fillna(df.mean(), inplace=True)
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Data Splitting
X = df.drop('Yield', axis=1)
y = df['Yield']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Model Training
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))
# model.save('yeild_val_loss_0.2203.keras')


ImportError: Traceback (most recent call last):
  File "C:\Users\raj\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
import joblib

# Save StandardScaler for later use
joblib.dump(scaler, "C:/Users/raj/Desktop/GDG/GDG/flask/scaler.pkl")

# Save feature names (order is important for predictions)
joblib.dump(X.columns.tolist(), "C:/Users/raj/Desktop/GDG/GDG/flask/feature_columns.pkl")